In [15]:
import warnings
import sympy as sp
from sympy import symbols
import random
import numpy as np
import pickle

"""
GPT prompt:
Given a sympy expression f, and its derivative f_dv, generate their values between -5 and +5, with interval 0.0001.
Then generate 100 random intervals (a, b) (-5 < a < b < 5). Store f(a), f(b), f_dv(a), f_dv(b), and 
values of f_dv(c) for a <= c <= b.

Then generate another 100 random intervals (a, b) (-5 < a < b < 5), so that f(a) == f(b). Store f(a), f(b), f_dv(a), f_dv(b), and 
values of f_dv(c) for a <= c <= b.
"""

import numpy as np
import sympy as sp
from collections import defaultdict


# Define an arbitrary function f(x); you can replace this with any other function
f_expr = sp.exp(-x**2) * sp.sin(5*x)


def generate_values(f_expr, verbose=False):
    # Define the symbolic variable and the function
    x = sp.symbols('x')
    f_dv_expr = sp.diff(f_expr, x)
    if verbose:
        display(f_expr)
        print("df/dx:")
        display(f_dv_expr)

    # Convert sympy expressions to numpy functions
    f = sp.lambdify(x, f_expr, modules=['numpy'])
    f_dv = sp.lambdify(x, f_dv_expr, modules=['numpy'])

    # Generate values between -5 and 5 with interval 0.0001
    x_values = np.arange(-5, 5.0001, 0.0001)
    f_values = f(x_values)
    print("Values generated")

    # Quantize the f(x) values to 4 decimal places (tolerance of 0.0001)
    tolerance = 0.0001
    quantized_f_values = np.round(f_values / tolerance) * tolerance

    # Build a hashtable mapping quantized f(x) values to x values
    f_x_dict = defaultdict(list)
    for f_val, x_val in zip(quantized_f_values, x_values):
        f_x_dict[f_val].append(x_val)
    print("Values put into dict")

    # Find intervals where |f(a) - f(b)| < 0.0001 and |b - a| >= 1
    equal_intervals = []
    max_intervals = 100

    num_attempt = 0
    x_keys = list(f_x_dict.keys())
    while len(equal_intervals) < max_intervals:
        num_attempt += 1
        if num_attempt > 100 * max_intervals:
            break
        f_val = random.choice(x_keys)
        x_list = f_x_dict[f_val]
        # Skip if less than two x values have this f(x) value
        if len(x_list) < 2:
            continue
        # Sort the x values
        x_list_sorted = sorted(x_list)
        n = len(x_list_sorted)
        # Use two pointers to find pairs where |b - a| >= 1
        left = random.randint(0, (n-1)//2)
        right = random.randint(left+1, n-1)
        # Move left pointer to ensure that x_list_sorted[right] - x_list_sorted[left] >= 1
        while x_list_sorted[right] - x_list_sorted[left] >= 1.0:
            num_attempt += 1
            if num_attempt > 100 * max_intervals:
                break
            a = x_list_sorted[left]
            b = x_list_sorted[right]
            # Compute f(a), f(b), f'(a), f'(b), and f'(c) between a and b
            f_a = f(a)
            f_b = f(b)
            f_dv_a = f_dv(a)
            f_dv_b = f_dv(b)
            c_values = np.arange(a, b, 0.0001)
            f_dv_c = f_dv(c_values)
            equal_intervals.append({
                'a': a, 'b': b,
                'f_a': f_a, 'f_b': f_b,
                'f_dv_a': f_dv_a, 'f_dv_b': f_dv_b,
                'f_dv_c': f_dv_c
            })
            break
            if len(equal_intervals) >= max_intervals:
                break
            left += 1
            if left == right:
                break

    # Now, 'equal_intervals' contains up to 100 intervals meeting the criteria

    # First 100 random intervals (unchanged)
    intervals = []
    for _ in range(100):
        a, b = sorted(random.uniform(-5, 5) for _ in range(2))
        if a == b:
            continue  # Ensure that a < b
        # Store the required values
        f_a = f(a)
        f_b = f(b)
        f_dv_a = f_dv(a)
        f_dv_b = f_dv(b)
        # Values of f_dv(c) for c between a and b
        c_values = np.arange(a, b, 0.0001)
        f_dv_c = f_dv(c_values)
        intervals.append({
            'a': a, 'b': b,
            'f_a': f_a, 'f_b': f_b,
            'f_dv_a': f_dv_a, 'f_dv_b': f_dv_b,
            'f_dv_c': f_dv_c
        })

    return intervals, equal_intervals

generate_values(f_expr)

Values generated
Values put into dict


([{'a': -0.42562615407628535,
   'b': 3.4358705742795124,
   'f_a': -0.7080469788250627,
   'f_b': -7.428730238865969e-06,
   'f_dv_a': -2.8091512097961293,
   'f_dv_b': 4.734377218222144e-05,
   'f_dv_c': array([-2.80915121e+00, -2.80766588e+00, -2.80617942e+00, ...,
           4.73731437e-05,  4.73632663e-05,  4.73533675e-05])},
  {'a': 0.49306028673397506,
   'b': 4.883659371626308,
   'f_a': 0.4908261207492568,
   'f_b': -2.8733799074145242e-11,
   'f_dv_a': -3.5419465922343356,
   'f_dv_b': 4.463056799075402e-10,
   'f_dv_c': array([-3.54194659e+00, -3.54262037e+00, -3.54329291e+00, ...,
           4.47863409e-10,  4.47342116e-10,  4.46821286e-10])},
  {'a': 3.631760445531235,
   'b': 3.864866362825218,
   'f_a': -1.1912779923112132e-06,
   'f_b': 1.489046786863513e-07,
   'f_dv_a': 1.5858729367528765e-05,
   'f_dv_b': 2.9752737933792444e-07,
   'f_dv_c': array([1.58587294e-05, 1.58451962e-05, 1.58316697e-05, ...,
          3.01139489e-07, 2.99384065e-07, 2.97631036e-07])},
  {'a'

In [6]:
# Build a hashtable mapping quantized f(x) values to x values
f_x_dict = defaultdict(list)
for f_val, x_val in zip(quantized_f_values, x_values):
    f_x_dict[f_val].append(x_val)

In [5]:
quantized_f_values.shape

(100001,)